In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year.

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work.

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR".
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s.
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required.
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor.
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization.
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't.
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE.
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things.
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short.
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after.
        <li> If you could use titles/bullet points I'd really appreciate it.
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people.
    </ul>
</ul>

In [4]:
#Load Data
df = pd.read_csv("Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

,count
Other technologies/programming languages you use often,
Javascript / Typescript,44
Python,37
SQL,31
"AWS, Docker",16
Kotlin,15
...,...
"Java / Scala, AWS, Kubernetes",1
"Java / Scala, AWS, Google Cloud, Kubernetes, Docker",1
"Go, Rust, Kubernetes",1


In [ ]:
df.head()

In [12]:
# --- Build the target we want to predict: Total yearly compensation (EUR) ---

salary_col = "Yearly brutto salary (without bonus and stocks) in EUR"
bonus_col  = "Yearly bonus + stocks in EUR"

# Bonus/stocks is often missing; treat missing bonus as 0 EUR
df[bonus_col] = df[bonus_col].fillna(0)

# Total compensation = base salary + bonus/stocks
df["total_comp"] = df[salary_col] + df[bonus_col]

# Quick check of the target distribution
df["total_comp"].describe()



,total_comp
count,1.253000e+03
mean,8.428361e+07
std,2.829182e+09
min,1.000100e+04
25%,6.000000e+04
50%,7.400000e+04
75%,9.100000e+04
max,1.000000e+11


In [30]:
# --- Survey data has fake entries like 9,999,999,999. Remove unrealistic totals. ---
# Keep compensation between 1,000 and 500,000 EUR (simple, reasonable bounds)

df = df[(df["total_comp"] >= 1000) & (df["total_comp"] <= 500000)]

df["total_comp"].describe()



,total_comp
count,1248.000000
mean,83717.752869
std,42329.673843
min,10001.000000
25%,60000.000000
50%,73450.000000
75%,91000.000000
max,400000.000000


In [32]:
# --- Reduce rare categories to "Other" to avoid huge one-hot encoding ---
# Any category appearing < 10 times becomes "Other"

rare_threshold = 10

high_card_cols = [
    "City",
    "Your main technology / programming language",
    "Other technologies/programming languages you use often"
]

for c in high_card_cols:
    if c in df.columns:
        vc = df[c].value_counts(dropna=False)
        rare = vc[vc < rare_threshold].index
        df[c] = df[c].where(~df[c].isin(rare), "Other")



In [33]:
# --- Build X (features) and y (target) ---
# IMPORTANT: Remove any salary/bonus/compensation columns from X to prevent leakage.
# Also remove Timestamp and any extra target-like columns such as TotalCompEUR.

drop_leak_cols = [
    "total_comp",
    "TotalCompEUR",  # <-- common leak if it exists
    salary_col,
    bonus_col,
    "Timestamp"
]

X = df.drop(columns=drop_leak_cols, errors="ignore")
y = df["total_comp"]

# Sanity checks (these should be False)
print("total_comp in X?", "total_comp" in X.columns)
print("TotalCompEUR in X?", "TotalCompEUR" in X.columns)
print("salary_col in X?", salary_col in X.columns)
print("bonus_col in X?", bonus_col in X.columns)

X.head()



total_comp in X? False
TotalCompEUR in X? False
salary_col in X? False
bonus_col in X? False


,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,Other,30,Full-time employee,Unlimited contract,English,51-100,Product
1,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,29.0,Male,Berlin,Software Engineer,12,6,Lead,Other,Other,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,37.0,Male,Berlin,Backend Developer,17,6,Senior,Other,Other,29,Full-time employee,Unlimited contract,English,101-1000,Product


In [44]:
# --- Build a simple pipeline: numeric + categorical preprocessing + Ridge regression ---

import numpy as np # Import numpy for sqrt

num_cols = X.select_dtypes(include="number").columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols),

        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    remainder="drop"
)

pipe = Pipeline([
    ("prep", preprocess),
    ("model", Ridge(alpha=1.0))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipe.fit(X_train, y_train)

pred_train = pipe.predict(X_train)
pred_test  = pipe.predict(X_test)

print("TRAIN R2:", r2_score(y_train, pred_train))
print("TRAIN RMSE:", np.sqrt(mean_squared_error(y_train, pred_train)))
print("TEST  R2:", r2_score(y_test, pred_test))
print("TEST  RMSE:", np.sqrt(mean_squared_error(y_test, pred_test)))

TRAIN R2: 0.547673477951679
TRAIN RMSE: 28134.24989359479
TEST  R2: 0.14006635845616366
TEST  RMSE: 40962.402677362545


In [41]:
# --- Feature Selection Activity ---
# Try dropping one messy column and compare test performance.

drop_col = "Other technologies/programming languages you use often"

X_fs = X.drop(columns=[drop_col], errors="ignore")

# Redefine num_cols and cat_cols based on X_fs
num_cols_fs = X_fs.select_dtypes(include="number").columns.tolist()
cat_cols_fs = [c for c in X_fs.columns if c not in num_cols_fs]

# Create a new preprocess transformer for X_fs
preprocess_fs = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols_fs),

        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols_fs),
    ],
    remainder="drop"
)

# Create a new pipeline for X_fs
pipe_fs = Pipeline([
    ("prep", preprocess_fs),
    ("model", Ridge(alpha=1.0))
])

X_train_fs, X_test_fs, y_train_fs, y_test_fs = train_test_split(
    X_fs, y, test_size=0.2, random_state=42
)

pipe_fs.fit(X_train_fs, y_train_fs)
pred_fs = pipe_fs.predict(X_test_fs)

print("AFTER DROPPING:", drop_col)
print("TEST R2:", r2_score(y_test_fs, pred_fs))
print("TEST RMSE:", np.sqrt(mean_squared_error(y_test_fs, pred_fs)))

AFTER DROPPING: Other technologies/programming languages you use often
TEST R2: 0.13510354317812245
TEST RMSE: 41080.432934695535


In [43]:
# --- Hyperparameter tuning (alpha for Ridge) ---

param_grid = {"model__alpha": [0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="r2")
grid.fit(X_train, y_train)

print("Best alpha:", grid.best_params_)
print("Best CV R2:", grid.best_score_)

best_model = grid.best_estimator_
best_pred = best_model.predict(X_test)

print("TUNED TEST R2:", r2_score(y_test, best_pred))
print("TUNED TEST RMSE:", np.sqrt(mean_squared_error(y_test, best_pred)))

Best alpha: {'model__alpha': 10}
Best CV R2: 0.1739030060057239
TUNED TEST R2: 0.20467106224485276
TUNED TEST RMSE: 39393.661160918826


# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes